In [29]:
#Importing the necessary libraries
import csv
import re
import numpy as np
import utility as ut
import features_final as f

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score

import pandas as pd
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score
from scipy import spatial


In [2]:
def clean(filename,filenameWrite):
    
    features = []
    with open(filename, "r", encoding="utf-8",newline='') as file:
        reader = csv.reader(file)
        for row in reader:
            #print len(row)
            title = row[2]
            description = row[6]
            cleanedDescription = ut.cleanDescription(description)
            #cleanedDescription = cleanText(cleanedDescription)
            cleanedTitle = ut.cleanText(title, False)
            
            # normalize price
            price = float(row[7])
            country = row[0]
            newPrice = ut.normalizePrice(price, country)
            
            cleanedRow = [row[0],row[1],cleanedTitle,row[3],row[4],row[5],cleanedDescription, newPrice,row[8]]
            features.append(cleanedRow)

    writer = csv.writer(open(filenameWrite, 'w',newline=''))
    writer.writerows(features)

In [11]:
#Should be done only once

print("Cleaning the text in the train data............")
clean('Data/training/data_train.csv','Data/training/data_train_clean.csv')

Cleaning the text in the train data............


In [2]:
#Load the word vectors

w2v_model = f.loadWordVectors('google_news_word_vectors/GoogleNews-vectors-negative300.bin')

Loading Word Vectors............
Loaded Word Vectors.............


In [3]:
#Pass the features that you want to test. The order of features are as given below
#word2VecFlag 
#lengthAndBooleanFlag
#tfidfLabelIntersectFlag
#numericFlag
#denseEntropySKUFlag


print("Computing Features............")
X = f.extract_features("Data/training/data_train_clean.csv",w2v_model,False,True,False,False,False)
X_input = X.astype(np.float)

print("Features Computed............")

y_input = np.loadtxt("data/training/conciseness_train.labels", dtype=float)

Computing Features............
Features Computed............


In [4]:
#Normalizing the features

X_std = (X_input - X_input.min(axis=0)) / (X_input.max(axis=0) - X_input.min(axis=0))
X_input = X_std
X_input = X_input[:,~np.all(np.isnan(X_input), axis=0)]

In [ ]:
#Pickle the files, load the pickled files
#Use this only if you want to pickle the features and use it later, else don't run this cell

import pickle

#pickle.dump(X_input, open( "X_input_numerical.p", "wb" ))
#pickle.dump(y_input, open( "y_input_numerical.p", "wb" ))

#Load either the X_input_binary or X_input_numerical depending on the features to be tested
#loaded_X_input = pickle.load( open( "X_input_binary.p", "rb" ) )
#loaded_y_input = pickle.load( open( "y_input_binary.p", "rb" ) )

#y_input = np.loadtxt("data/training/clarity_train.labels", dtype=float)

X_input = loaded_X_input

In [ ]:
X_input.shape[1]

In [ ]:
### DEEEP LEARNING!!!!!!!!!!!!
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import optimizers
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import LSTM

#Using a 4-fold to get a good train-test split

kFoldResults = ut.splitKFold(X_input,y_input,4)

trainRMSE = []
testRMSE = []
trainF1 = []
testF1 = []
testPS = []
testRAS = []
testAcc = []
testRecall = []
testAvgPS = []

for resultGenerator in kFoldResults:
    
    batch_size = 32
    filters = 150
    filter_sizes = [2,3,5]
    hidden_dims1 = 200
    hidden_dims2 = 100
    num_epochs = 10
    sgd = optimizers.SGD(lr=1.1)
    model = Sequential()

    #First Dense Layer
    model.add(Dense(600,input_dim=X_input.shape[1],kernel_initializer="random_uniform",activation="elu"))
    model.add(Dropout(0.2))
    
    #Second Dense Layer
    model.add(Dense(300,kernel_initializer="random_uniform",activation="elu"))
    model.add(Dropout(0.4))
    
    #Third Dense Layer
    model.add(Dense(200,kernel_initializer="random_uniform",activation="elu"))
    model.add(Dropout(0.4))
    
    #Fourth Dense Layer
    model.add(Dense(100,kernel_initializer="random_uniform",activation="relu"))
    model.add(Dropout(0.4))
    
    #Final Output Layer
    model.add(Dense(1,kernel_initializer="random_uniform",activation="sigmoid"))
    
    model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    

    trainIndices = resultGenerator[0]
    testIndices = resultGenerator[1]
    
    X_train = list()
    y_train=list()
    X_test=list()
    y_test=list()
    
    for trainRow in trainIndices:
        X_train.append(X_input[trainRow])
        y_train.append(y_input[trainRow])

    X_train = np.asarray(X_train)
    y_train = np.asarray(y_train)
    
    # Model training
    print("Training the Model............")
    
    
    model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size)    
    
    
    print("Features Computed for Testing data............")
    X_test = []
    y_test = []
        
    for testRow in testIndices:
        X_test.append(X_input[testRow])
        y_test.append(y_input[testRow])

    X_test = np.array(X_test)
    y_test_list = y_test
    y_test = np.array(y_test)
    
    
    #Evaluate the model
    loss_and_metrics = model.evaluate(X_test, y_test, batch_size=128)
    predicted_results = model.predict_classes(X_test, batch_size=128)
    
    
    #Get the score metrics
    acc = accuracy_score(y_test, predicted_results)    
    f1 = f1_score(y_test,predicted_results)
    ras = roc_auc_score(y_test,predicted_results)
    ps = precision_score(y_test,predicted_results)
    recall = recall_score(y_test,predicted_results)
    average_precision = average_precision_score(y_test,predicted_results)
    
    testAcc.append(acc)
    testRAS.append(ras)
    testF1.append(f1)
    testPS.append(ps)
    testRecall.append(recall)
    testAvgPS.append(average_precision)
    
    print(acc)
    print(f1)
    print(ras)
    print(ps)
    print(recall)
    print(average_precision)   
    

In [ ]:
model.summary()

In [ ]:
print('Average')
print(np.mean(testAcc))
print(np.mean(testF1))
print(np.mean(testRAS))
print(np.mean(testPS))
print(np.mean(testRecall))
print(np.mean(testAvgPS))